# Firstly, import the necessary libraries.
If you are using Kaggle, you might have to use **sys.path.append** for the lib folder.

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
sys.path.append('/kaggle/input/libreunet/lib')
sys.path.append('../input/reunetdata64/REUNETData64')
sys.path.append('../input/reunet64model')

import loaders, modules
from trainandtest import TrainModel

# Dataloader

In [ ]:
Radio_train = loaders.RadioUNet_s_sprseIRT4(phase="train", simulation="IRT4", cityMap="complete",fix_samples = 300,n_iterations=10, dir_dataset='/kaggle/input/reunetdataset/REUNETData/')
Radio_val = loaders.RadioUNet_s_sprseIRT4(phase="val", simulation="IRT4", cityMap="complete",fix_samples = 300,n_iterations=10, dir_dataset='/kaggle/input/reunetdataset/REUNETData/')
Radio_test = loaders.RadioUNet_s_sprseIRT4(phase="test", simulation="IRT4", cityMap="complete",fix_samples = 300,n_iterations=10, dir_dataset='/kaggle/input/reunetdataset/REUNETData/')
# Radio_test = loaders.RadioUNet_s_sprseIRT4(phase="custom",ind1=0, ind2=100, simulation="IRT4", cityMap="complete",fix_samples = 300,n_iterations=10)

image_datasets = {
    'train': Radio_train, 'val': Radio_val
}

batch_size = 15

dataloaders = {
    'train': DataLoader(Radio_train, batch_size=batch_size, shuffle=False, num_workers=1),
    'val': DataLoader(Radio_val, batch_size=batch_size, shuffle=False, num_workers=1)
}

# Model Load

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = modules.RadioWNet(inputs=2, phase="secondU")
model.load_state_dict(torch.load('../input/reunet64model/Trained_Model_secondU_AI_Stack_e750.pt',map_location=device))
# model.load_state_dict(torch.load('../input/trainmodelsecondu5e100updated/Trained_Model_secondU_5_updated_e100.pt', map_location = device))

# Train Model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import os
try:
  os.mkdir('NewRMModel/') 
except OSError as error:
  print(error)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)


# range_measurements = list(range(10,150,10)) + list(range(150,800,50))
range_measurements = list(range(600,800,50))


for sensor_measurements in range_measurements:
    optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)
    
    Radio_train = loaders.RadioUNet_s_sprseIRT4(phase="train", simulation="IRT4", cityMap="complete",fix_samples = sensor_measurements,n_iterations=10, dir_dataset='../input/reunetdata64/REUNETData64/')
    Radio_val = loaders.RadioUNet_s_sprseIRT4(phase="val", simulation="IRT4", cityMap="complete",fix_samples = sensor_measurements,n_iterations=10, dir_dataset='../input/reunetdata64/REUNETData64/')
    image_datasets = {
        'train': Radio_train, 'val': Radio_val
    }

    batch_size = 15

    dataloaders = {
        'train': DataLoader(Radio_train, batch_size=batch_size, shuffle=False, num_workers=1),
        'val': DataLoader(Radio_val, batch_size=batch_size, shuffle=False, num_workers=1)
    }
    Training_loaddata = TrainModel(dataloaders)

    model = Training_loaddata.train_model(model, optimizer_ft, exp_lr_scheduler, targetType = 'dense', num_epochs=11,WNetPhase="secondU",train_log=True)

    #Save the model in your folder as a .pt file
    torch.save(model.state_dict(), f'NewRMModel/Trained_Model_secondU_AI_Stack_e{sensor_measurements}.pt')
    
    
    
#     for epoch in range(2,6):
#     model.load_state_dict(torch.load(f'NewRMModel/Trained_Model_SecondU_{epoch - 1}.pt'))
#     model = Training_loaddata.train_model(model, optimizer_ft, exp_lr_scheduler, targetType = 'dense', num_epochs=10,WNetPhase="secondU",train_log=True)

    #Save the model in your folder as a .pt file
#     torch.save(model.state_dict(), f'NewRMModel/Trained_Model_SecondU_{epoch}.pt')

# Test the Model

In [ ]:
from skimage.metrics import structural_similarity, mean_squared_error
import statistics
import seaborn as sns
maps_inds=np.arange(0,700,1,dtype=np.int16)
#Standard determenistic "random" shuffle of the maps:
np.random.seed(42)
np.random.shuffle(maps_inds)
from PIL import Image
from time import perf_counter
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

try:
    os.mkdir('Examples/') 
except OSError as error:
    print(error)
    

total_range = list(range(10,150,10)) + list(range(150,800,50)) #This is the testing range that is fixed at all times. Don't change it anymore

for starting in [0,600]:
    mse_mother = {}
    time_mother = {}
    for sensors in total_range:
          for mapp in range(1,100):
                start = perf_counter()
                name00=str(mapp)
                name0=str(maps_inds[0+mapp])
                Radio_test2 = loaders.RadioUNet_s_sprseIRT4(maps_inds=maps_inds, phase="custom", 
                                                        ind1=starting+mapp, ind2=starting+mapp, 
                                                        simulation="IRT4", cityMap="complete",fix_samples = sensors, n_iterations=1,numTx=1,data_samples=sensors,
                                                        dir_dataset='../input/reunetdata128/REUNETData128/')   
                ii=0
                for inputs, target in DataLoader(Radio_test2, batch_size=1, shuffle=False, num_workers=0):
                    ii=ii+1
                    inputs = inputs.to(device)
                    target = target.to(device)
                    pred1, pred = model(inputs)
                    pred = (pred.detach().cpu().numpy()).astype(np.uint8)
                    builds=inputs.detach().cpu().numpy()
                    builds=builds[0][0]
                    inputs = (inputs.detach().cpu().numpy()).astype(np.uint8)
                    indB=builds!=0
                    im=np.zeros([256,256,3])
                    #imT=np.zeros([256,256,3])
                    im[:,:,0] = (pred[0][0])
                    im[:,:,1] = (pred[0][0])
                    im[:,:,2] = (pred[0][0])
                    im[indB,2]=130 # Add buildings
                    #imT=im
                    ind0=inputs[0][1]!=0
                    ind1=inputs[0][1]>=100 
                    # im[ind0,1]=0
                    # im[ind0,0]=255
                    # im[ind0,2]=20
                    # im[ind1,0]=80
                    # im[ind1,2]=0
                    im = Image.fromarray(im.astype(np.uint8))
                    name=str(ii)
                    #file_name="buildC_carsN_simulD_DMP_samplesN/%spredict.png" %name
                    file_name="Examples/" +name00+ "_"+name0+ "_"+ name + "IRT4_predict.png" 
                    im.save(file_name)
                    #file_name="%spredict.png" %name
                    im = Image.fromarray(255*inputs[0][1])
                    file_name="Examples/" +name00+ "_"+name0+ "_"+ name + "IRT4_samples.png" 
                    im.save(file_name)
                    Gtruth = (target.detach().cpu().numpy() ).astype(np.uint8)
                    im=np.zeros([256,256,3])
                    im[:,:,0]=Gtruth[0][0]
                    im[:,:,1]=Gtruth[0][0]
                    im[:,:,2]=Gtruth[0][0]
                    im[indB,2]=130
                    im = Image.fromarray(im.astype(np.uint8))
                    #file_name="buildC_carsN_simulD_DMP_samplesN/%starget.png" %name
                    file_name="Examples/" +name00+ "_"+name0+ "_"+ name + "IRT4_target.png"
                    #file_name="%starget.png" %name
                    im.save(file_name)
                    pred = pred*(99.16/255) - 124
                    Gtruth = Gtruth*(99.16/255) - 124
                    # new = sns.heatmap(pred[0][0],cmap="Greys_r")
                    # new
                    mqe = mean_squared_error(pred, Gtruth)
                    try:
                        mse_mother[sensors].append(mqe)
                        time_mother[sensors].append(perf_counter()-start)
                    except KeyError:
                        mse_mother[sensors] = [mqe]
                        time_mother[sensors] = [perf_counter()-start]

    mse_df = pd.DataFrame.from_dict(mse_mother, orient='index', columns=list(range(1,100))).transpose()
    time_df = pd.DataFrame.from_dict(time_mother, orient='index', columns=list(range(1,100))).transpose()
    test_train_string = 'test'if starting == 600 else 'train'
    mse_df.to_csv(f'{datetime.now().strftime("%y%m%d_%H%M")}_testlog_mse_{test_train_string}.csv')
    time_df.to_csv(f'{datetime.now().strftime("%y%m%d_%H%M")}_testlog_time_{test_train_string}.csv')




